# ML Reverse Lecture

In this lab we will look at some examples on how to use ML for classification tasks over a music dataset.

Specifically, we will use a Kaggle dataset with 218k songs [fetched from Spotify](https://github.com/tgel0/spotify-data). It has a schema that is almost identical to the one we used for lab 2, but with two additional columns: `genre`, and `popularity`. Schema columns are described in the [Spotify API docs](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

First, we will do some data exploration over the top 1k most popular songs in the dataset. Our goal in this phase is to understand some of the characteristics that these most popular songs have in common. We'll use visualization techniques and unsupervised ML (k-means clustering, PCA) to better understand the relationship between some of the features.

Next, we'll try out a genre prediction task on the entire 218k songs dataset. We'll use supervised ML (classification) for this, and give you a chance to try out the same problem using regression.

Finally, we'll see if we can use what we learned during the genre prediction task to try and predict song popularity.


## Running this notebook

To execute code from a cell, you can either click `Run` at the top, or type `shift+Enter` after clicking a cell.  You can either run the entire notebook (`Restart & Run All` from the `Kernel` drop-down), or run each cell individually.  If you choose the latter, note that it is important that you run cells in order, as later cells depend on earlier ones. And to be able to `Run All` successfully, you'll have to write code for answering some of the questions in the notebook.

Once you open your notebook on the browser, and check that the cells are rendering correctly (e.g., try running the "Python packages" cell below), we're good to go from there.

## Python packages we'll need

First, import the python packages we'll be using for the lab:

In [ ]:
# Dataframes.
import pandas as pd
import numpy as np

# Plotting.
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
sns.set_style('whitegrid')

# ML.
from sklearn import tree
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Ignore warnings.
import warnings
warnings.simplefilter("ignore")

## Part 1: data exploration

First, we load our dataset into a pandas dataframe. Next, we store the top 1k most popular songs in a separate `top1k` dataframe.

**Q1: Filter `df` for the top 1000 songs with largest `popularity` values, and store it on a `top1k` dataframe below.**

In [ ]:
df = pd.read_csv('data/spotify_songs.csv')

# Q1: YOUR CODE GOES HERE.
# top1k = (...)

Let's make sure that loading our data worked as we expect. A natural way to do that is to inspect the first few entries with [`head()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html):

In [ ]:
df.head()

We can see that most of these columns are familiar to us, as they were present in our `top2018.csv` dataset from lab 2. The additional columns here are only `genre` and `popularity`.

Let's examine the data types for each of the columns using [`info()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html):

In [ ]:
df.info()

The output of `df.info()` tells us that we have:
- 7 `object` columns: (`genre`, `artist_name`, `track_name`, `track_id`, `key`, and `time_signature`)
- 2 `int64` columns (`popularity` and `duration_ms`)
- 9 `float64` (remaining columns)

We can also take a look at aggregate stats for each of the numerical columns (both discrete `int64`, and continuous `float64`) the dataset has by running [`describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html):

In [ ]:
df.describe()

Except for `popularity`, `duration_ms`, `loudness`, and `tempo`, most of the numerical features are already scaled between 0.0 and 1.0.

**Q2: Looking at the documentation for [describe()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), what's a command to describe categorical features? Why is the output different for categorical vs numerical features?**

In [ ]:
# Q2: YOUR CODE GOES HERE
# df.describe(...)

Next, let's check whether this dataset has missing data in the form of `null`s:

**Q3: How would you check whether your data set has null values in any of its columns? Did you find any?**

In [ ]:
# Q3: YOUR CODE GOES HERE.
# df.(...)

Let's examine the dataset's [correlation matrix](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) to see how column values co-occur with each other. This can help us spot features that may be predictive of other features in the dataset:

In [ ]:
corr = df.corr()
corr.style.background_gradient().set_precision(2)

It seems that `energy` and `loudness` is the pair of features with the strongest positive correlation, followed by `valence` and `danceability`. We also see strong negative correlation between `acousticness` and a few other features. Finally, some of these have a medium to strong correlation with popularity.

**Q4: Why don't we see `genre`, `key`, and `time signature` on the correlation matrix? What could we do to address this?**

If we look at the [documentation for these features](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/), we see that `energy` is defined as a "perceptual measure of intensity and activity".  The example they give for this is death metal (high energy) vs a Bach prelude (low energy). Similarly, `valence` is a score between `0.0` and `1.0` indicating how positive the song is. So it makes sense that the `energy` would be correlated with `loudness`, and `valence` with `danceability`.

Now let's examine the distribution values for some of the features we mention above. In particular, let's look at the `top1k` subset, as we may have a better intuition for popular songs. For example, are the `top1k` songs more biased towards high `danceability`?

We can use [`seaborn's distplot()`](http://seaborn.pydata.org/generated/seaborn.distplot.html) to plot a histogram for the `danceability` column:

In [ ]:
sns.distplot(top1k['danceability']).set_title('Danceability distribution for top 1k songs')

How about `energy` and `acousticness`? From the API docs: acousticness is a score between 0.0 and 1.0 that indicates Spotify's confidence that the song is "acoustic", e.g., live performances would have a score close to or equal to 1.0. 

**Q5: Plot the distribution for energy and acousticness below.  What is the mean for each?**

In [ ]:
# Q5: YOUR CODE GOES HERE
# (...)

**Q6: Discuss with the person next to you: what have we learned so far on the relationship between top1k songs and these 3 features above? How do you think this may affect our `popularity` prediction task?**

**In other words, what kind of song would you write if you were trying to make it to the top as a musician, just based on these 3 features alone?**

## Part 2: unsupervised ML with k-means clustering

What if instead examining feature by feature we were to try and group songs by how similar they are in terms of these features?

If you think of each song as a vector in a multidimensional space, where each dimension is a feature in the dataset, we can use co-sine similarity as a "similarity" metric to group together (or "cluster") songs that are similar.  And using the same idea, we can also "profile" each group, or "cluster".

An unsupervised ML algorithm that can help us here is [K-means clustering](https://en.wikipedia.org/wiki/K-means_clustering). `K` stands for the number of clusters (or groups) you want to have. The term `mean` refers to the method you use to assign each element in the dataset to a target cluster: by choosing the nearest mean (also known as the "centroid" element, representing the cluster) to that element.

Specifically, for our case, given a set of songs where each song is a `d-`dimensional real vector, `k-`means clustering aims to partition the set of songs into `k` clusters so as to minimize the within-cluster sum of squares (aka intra-cluster variance).

At this point we don't know yet how many clusters we'll need, but let's give it a try using some of the features we've seen so far, plus other characteristics such as `speechiness` (i.e., amount of vocals in the song), and `instrumentalness`.

In [ ]:
# We exclude non-numerical features, and those that are too fine-grained, such as "key", or too coarse-grained,
# such as "mode".
df_numerical = top1k.select_dtypes(exclude=[np.object])
cluster_features = ['acousticness', 'danceability', 'energy', 'speechiness', 'instrumentalness']
df_clusters = df_numerical[cluster_features]

# numpy array with features
X = np.array(df_clusters)

# Q7: Do we need this scaling code?
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Hint: does this help you decide?
#top1k[cluster_features].describe()

How many clusters should we use? If we pick too few, we get too many songs that don't quite "fit in" with that cluster (i.e., intra-cluster variance, or error, is too large). If we pick too many, error is minimized, but our clusters are too small.

To figure out the "sweet spot", we can plot how the error decays as a function of the number of clusters we use.  This will give us a "elbow" shaped curve, and we can use the point at which the curve turns "smooth" as our number of clusters:

In [ ]:
dist = []
for k in range(1, 10):
    km = KMeans(n_clusters=k, init='k-means++', random_state=1337)
    km = km.fit(X)
    dist.append(km.inertia_)
    
plt.plot(range(1, 10), dist, 'bx-')
plt.xlabel('k')
plt.ylabel('Variance')
plt.title('Elbow Method for choosing k')
plt.show()

**Q7: We used co-sine distance between songs (represented as n-dimensional vectors) to cluster them a 5-D space (n=5 features).  A [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) standardizes features by removing the mean and scaling to unit variance.  Do we need to use that scaler in our features (see code 2 cells above)? Plot the elbow method above both with and without scaler to see the difference.**

We can see from the "Elbow Method" curve above that at `k=4` our curve starts turning smooth. We can use either `k=4` or `k=5` as our number of clusters for k-means below. We'll go with `k=5`:

In [ ]:
# k-means clustering
kmeans = KMeans(n_clusters=5, random_state=1337)
kmeans.fit(X)
y = kmeans.predict(X)

We can now try and visualize the clusters in a 2-D space:

In [ ]:
# visualize the clusters
centers = kmeans.cluster_centers_
print('Features:\n', cluster_features)
print('Centroids:\n', centers)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5);

The centroid values for each dimension tells us some information on what the cluster profiles are, e.g., the last cluster is high on acousticness and low on both danceability and energy, so it's likely capturing live performances.

But otherwise, as it turns out, directly visualizing the clusters in a 2-D space isn't so helpful for us at this point.

First, we're using a 5-D dimensional space for clustering, so without doing something smart about how we reduce the number of dimensions when polotting, we're losing a lot of information on the visualization above.  Second, most songs in this dataset are highly biased towards one end of each of some features (e.g., we have a lot more danceable songs than we have non-danceable ones), so we'll have a lot of clusters fairly close to each other.

We have a few options for dimensionality reduction that can help us here. The first one is Principal Component Analysis ([PCA](https://en.wikipedia.org/wiki/Principal_component_analysis)). Using PCA, we can tell what is the smallest set of features that contain the most information about our dataset:

In [ ]:
pca = PCA(n_components=len(cluster_features))
pca_components = pca.fit_transform(X)

# plot the PCA variances
pca_features = range(pca.n_components_)
plt.bar(pca_features, pca.explained_variance_ratio_)
plt.xlabel('Principal components')
plt.ylabel('Explained variance (%)')
plt.xticks(pca_features)

# save pca components in a df
pca_components = pd.DataFrame(pca_components, columns = ['PC0', 'PC1', 'PC2', 'PC3', 'PC4'])

We can see from the plot above that after the second PC, the gain in information is reduced. That is, the first component contributes almost 30% of the information, and the second one contributes over 25%. So if we use just the first 2 PCs, we already have close to 60% of the data being preserved. If we kept the first 3 PCs, we get closer to 80%.

Let's try and plot the k-means clusters using PCA instead, and see if that makes it easier to visualize the clusters:

In [ ]:
# k-means clustering on first 2 PCA components
X_pca = pca_components
kmeans_pca = KMeans(n_clusters=5, random_state=31337)
kmeans_pca.fit(X_pca)
y_pca = kmeans_pca.predict(X_pca)

# visualize the clusters
centers_pca = kmeans_pca.cluster_centers_
plt.scatter(X_pca['PC0'], X_pca['PC1'], c=y_pca, s=50, cmap='viridis')
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', s=200, alpha=0.5);
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# if you're curious, inspect the min and max values for the PC dimensions
# X_pca[['PC0', 'PC1']].describe()

As we can see above, using k-means clustering after PCA, and plotting our clusters with the first 2 PCs as dimensions, helps a bit. Specifically, intra-cluster variance is smaller, and we can see that the centroids are not that far apart from each other (likely indicating that songs in the top1k are not that different from each other). But we can still improve this visualization.

A more powerful technique for dimensionality reduction is t-Distributed Stochastic Neighbor Embedding ([t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)). We'll leave for you to **try on your own** by following this [t-SNE vs PCA tutorial](https://www.datacamp.com/community/tutorials/introduction-t-sne), which has several example comparisons between the results you get with PCA vs t-SNE for visualization.

## Part 3: supervised ML for prediction tasks

For this last part, we'll focus on two prediction tasks.

In the first task, we'll try and predict genre for each song. We'll first try and see if we can do that on the `top1k` songs, and then with the entire dataset.

In the second task, we'll try to predict song popularity, but using a larger dataset with features similar to those of the dataset above.

### a) genre prediction

As mentioned above, the dataset we've been using so far has been augmented with genre information (stored in the `genre` column). We'll use values in that column as labels for training a supervised ML algorithm to predict song genre.

And unlike in k-means clustering, here we'll use all features as input. We start by scaling our dataset again, follow that by splitting into our data into training and test datasets:

In [ ]:
# use our entire set of numerical features, instead of just the 5 we used for k-means clustering:
df_numerical = top1k.select_dtypes(exclude=[np.object])
X = np.array(df_numerical)

# scaling won't make a difference for classifiers, but we leave it here in case you want to try other models
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# the features and the label we'll use for training our supervised algo:
features = X
labels = top1k['genre']

# split our data into training and test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=31337)

We then train our [decision tree classifier](https://scikit-learn.org/stable/modules/tree.html):

In [ ]:
# train a decision tree on it
genre_tree = DecisionTreeClassifier(random_state=31337)
genre_tree.fit(train_features, train_labels)

# predict labels on test data
pred_labels_tree = genre_tree.predict(test_features)

Let's see how well the tree did on our genre prediction task:

In [ ]:
tree_results = classification_report(test_labels, pred_labels_tree)
print('Decision Tree Results:\n',  tree_results)

That's pretty bad: our weighted average precision is as bad as random.  This is probably because this dataset doesn't have enough samples of each genre (there are only 1000 songs in total). As expected, we have a bias only towards genres in the top 1000.

**Q8: Did we pick the wrong model? Try a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) instead below, and see if we can get better results:**

In [ ]:
# Q8: YOUR CODE GOES HERE.
# (...)

Another problem here is that not only do we have a small dataset, but we also have a large number of genres relative to the size of the dataset.

Let's try and see if we can do better with more samples by using the larger dataset, and also reducing it down to prediction of only 3 mainstream genres:

**Q9: How many samples do we have per genre on the larger dataset (`df` pd dataframe)?** 

In [ ]:
# Q9: YOUR CODE GOES HERE.
# df.(...)

This may be enough, but as we can see the number of samples is not exactly balanced for each genre, so we may still get some bias.  Also, it's a fair amount of genres, so let's pick 3 common ones and see how we can do at that task. We'll go with "Hip-Hop", "Rock", and "Jazz", which should be different enough from each other:

In [ ]:
hip_hop = df.loc[df['genre'] == 'Hip-Hop']
rock = df.loc[df['genre'] == 'Rock']
jazz = df.loc[df['genre'] == 'Jazz']

# make it equal number of songs for each
hip_hop = hip_hop.sample(n=len(rock), random_state=31337)
jazz = jazz.sample(n=len(rock), random_state=31337)
df = pd.concat([rock, hip_hop, jazz])

And train our forest on this larger dataset:

In [ ]:
# select only numerical features
df_numerical = df.select_dtypes(exclude=[np.object])
X = np.array(df_numerical)

# scaling won't make a difference for classifiers, but we leave it here in case you want to try other models
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# features and label
features = X
labels = df['genre']

# split our data into training and test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=31337)

# train a RFC on it
genre_forest = RandomForestClassifier(random_state=31337)
genre_forest.fit(train_features, train_labels)

# predict labels on test data
pred_labels_forest = genre_forest.predict(test_features)

Let's see how we did this time around:

In [ ]:
genre_forest_results = classification_report(test_labels, pred_labels_forest)
print('RFC Results:\n',  genre_forest_results)

This time around we did a lot better: 92% precision for Jazz, and an average precision of 88% for all 3 genres.

**Q10: To try on your own: train a [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) with different values of `max_depth` and use `tree.plot()` to [`plot it`](https://scikit-learn.org/stable/modules/tree.html).**

**What are some pros/cons of using a decision tree vs a random forest classifier?**

In [ ]:
# Q10: YOUR CODE GOES HERE

### b) popularity prediction

Let's use what we learned above to try out another prediction task: song popularity. First, we'll do some feature engineering:

In [ ]:
# make sure we start with a clean slate
df = pd.read_csv('data/spotify_songs.csv')

# hot-encode "mode" categorical column
df.loc[df['mode'] == 'Major', 'mode'] = 1
df.loc[df['mode'] == 'Minor', 'mode'] = 0

# hot-encode "key" categorical columns, i.e.,
# A -> 0, A# -> 1, etc.
keys = df['key'].unique()
for i in range(len(keys)):
    df.loc[df['key'] == keys[i], 'key'] = i

# ditto for time signatures
signatures = df['time_signature'].unique()
for i in range(len(signatures)):
    df.loc[df['time_signature'] == signatures[i], 'time_signature'] = i

We'll pose our popularity prediction task as a binary decision between "popular" and "unpopular". Our data is exponentially distributed in terms of popularity (**how can we check that?**), so we'll choose a cutoff at the 75th percentile to split our songs at:

In [ ]:
# convert popularity from [1, 100] to "popular" or "not popular" by splitting below and above the 75th %
pop_75th = np.percentile(df['popularity'], 75)
df.loc[df['popularity'] < pop_75th, 'popularity'] = 0 
df.loc[df['popularity'] >= pop_75th, 'popularity'] = 1

And as we've done before, split into train and test datasets. We'll use these to train a [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):

In [ ]:
# select only numerical features
df_numerical = df.select_dtypes(exclude=[np.object])

# remove our label from features
df_features = df_numerical.drop(['popularity'], axis=1)

# standard scale
X = np.array(df_features)
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# features and label
features = X
labels = df['popularity']

# split our data into training and test, reserving 20% of rows for test set
train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                            labels, test_size=0.2, random_state=31337)

And let's train a few other ML algos (some more powerful) on these:

In [ ]:
# train RFC
rfc = RandomForestClassifier(random_state=31337)
rfc.fit(train_features, train_labels)

# predict labels on test data
pred_labels_rfc = rfc.predict(test_features)
accuracy_rfc = accuracy_score(test_labels, pred_labels_rfc)
print('RFC Accuracy: ' + str(accuracy_rfc))

auc_rfc = roc_auc_score(test_labels, pred_labels_rfc)
print('RFC AUC: ' + str(auc_rfc))

Not bad!

Now go forth and **try out for yourself with other models, and compare their performance**. In particular, we recommend you try out `xgboost`'s [`XGBClassifier`](https://xgboost.readthedocs.io/en/latest/python/index.html), [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and [LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) and see how those do on the same task. Do they beat the RFC above?

You can also try out regression on the `popularity` ranks, instead of the binary decision classification problem we formulated above.

In [ ]:
# Q11: YOUR CODE GOES HERE
# Try out xgboost's XGBClassifier as well as try and formulate the problem as a regression on popularity ranks.